In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Conv1D,Conv2D
#from keras.layers.convolutional import Convolution1D,Conv1D
# If using TensorFlow 2.x with Keras API
from tensorflow.keras.layers import Conv2D, MaxPooling1D

#from keras.layers.convolutional import MaxPooling1D
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from keras import backend as K
from sklearn import preprocessing
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam, SGD
#from sgdr import SGDRScheduler

In [22]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import Callback

class SGDR(Callback):
    def __init__(self, max_lr, min_lr, restart_interval, decay_factor=0.5):
        super(SGDR, self).__init__()
        self.max_lr = max_lr
        self.min_lr = min_lr
        self.restart_interval = restart_interval
        self.decay_factor = decay_factor
        self.iterations = 0

    def steps_per_epoch(self, epoch, logs=None):
        # Cycle calculation
        cycle = np.floor(1 + self.iterations / (2 * self.restart_interval))
        x = np.abs(self.iterations / self.restart_interval - 2 * cycle + 1)
        lr = self.min_lr + (self.max_lr - self.min_lr) * np.maximum(0, (1 - x))

        # Set the learning rate for the optimizer
        tf.keras.backend.set_value(self.model.optimizer.lr, lr)

        # Increment the iteration
        self.iterations += 1

        # Print the learning rate
        print(f'\nEpoch {epoch+1}: Learning rate is {lr:.6f}')


In [ ]:
#pip install keras
!pip install sgdr



In [89]:
fields = ['Flow Bytes/s', 'Fwd IAT Mean','Flow IAT Mean','Flow Packets/s','Fwd IAT Max','Flow IAT Max',
          'Bwd IAT Max','Flow IAT Std','Bwd IAT Mean','Bwd IAT Min','Fwd IAT Std','Bwd IAT Std','Fwd IAT Min',
          'Flow IAT Min','label']

#fields = ['Flow Bytes/s', 'Flow Duration','Flow IAT Mean','Flow Packets/s','Flow IAT Max','Flow IAT Std','Flow IAT Min','label']

#fields = ['Flow IAT Min','Bwd IAT Std','Bwd IAT Mean','Bwd IAT Max','label']


df = pd.read_csv("DATATORNONTOR.csv", skipinitialspace=True, usecols=fields)


df.values

array([[0.0, 4597.7011494253, 435.0, ..., 0, 0, 'nonTOR'],
       [0.0, 7722.0077220077, 259.0, ..., 0, 0, 'nonTOR'],
       [0.0, 2244.6689113356, 891.0, ..., 0, 0, 'nonTOR'],
       ...,
       [inf, inf, 0.0, ..., 0, 0, 'TOR'],
       [nan, inf, 0.0, ..., 0, 0, 'nonTOR'],
       [nan, inf, 0.0, ..., 0, 0, 'nonTOR']], dtype=object)

In [176]:
import scipy.io.arff
import pandas as pd

# Load the ARFF file
########30s###TOR non TOR###data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TORnTOR30secTraining.arff')#TORnTOR30secTraining
##data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-60s-TOR-NonTOR-85.arff')#TORnTOR30secTraining
##data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-30s-TORNonTOR-85.arff')#TORnTOR30secTraining
##data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-15s-TOR-NonTOR-85.arff')#TORnTOR30secTraining
data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-120s-TOR-NonTOR-85.arff')#TORnTOR30secTraining



# Convert to pandas DataFrame for easier manipulation
df = pd.DataFrame(data)

# Convert any byte-like strings to regular strings (if needed)
df = df.apply(lambda x: x.str.decode('utf-8') if x.dtype == 'O' else x)

# Define the columns you're interested in
fields = ['min_biat','min_fiat','max_biat','max_fiat','flowBytesPerSecond','mean_flowiat',
          'flowPktsPerSecond','mean_fiat','max_flowiat','mean_biat','std_flowiat','min_flowiat','duration','class1']

# Select only the specified columns
df= df[fields]

# Show the first few rows of the selected columns
df.head()


<>:9: SyntaxWarning: invalid escape sequence '\T'
<>:9: SyntaxWarning: invalid escape sequence '\T'
C:\Users\tm120a\AppData\Local\Temp\ipykernel_35420\1617347956.py:9: SyntaxWarning: invalid escape sequence '\T'
  data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-120s-TOR-NonTOR-85.arff')#TORnTOR30secTraining


,min_biat,min_fiat,max_biat,max_fiat,flowBytesPerSecond,mean_flowiat,flowPktsPerSecond,mean_fiat,max_flowiat,mean_biat,std_flowiat,min_flowiat,duration,class1
0,35149142.0,35188437.0,5.578672e+05,4.742976e+05,1411.640099,2.559010e+05,3.916293,2.502257e+06,35137650.0,2.693576e+06,1.839548e+06,4.0,117202678.0,NONTOR
1,35022877.0,35208950.0,5.410036e+05,5.108043e+05,1731.104649,2.622349e+05,3.821737,2.582052e+06,35022877.0,2.634447e+06,1.849452e+06,4.0,119579137.0,NONTOR
2,25978548.0,26116331.0,7.503962e+05,6.873223e+05,1238.133672,3.570551e+05,2.815992,3.213452e+06,25978548.0,3.322049e+06,2.313499e+06,5.0,65341087.0,NONTOR
3,30001305.0,30001029.0,3.000000e+07,3.000000e+07,6.083375,1.500000e+07,0.075001,8.268222e+02,29944585.0,1.154763e+03,1.600000e+07,54824.0,119999180.0,NONTOR
4,30000507.0,30000886.0,3.000000e+07,3.000000e+07,3.250021,1.500000e+07,0.075000,7.304966e+02,29941980.0,6.825712e+02,1.600000e+07,57471.0,119999214.0,NONTOR


In [177]:
len(df.columns)
print(fields)

['min_biat', 'min_fiat', 'max_biat', 'max_fiat', 'flowBytesPerSecond', 'mean_flowiat', 'flowPktsPerSecond', 'mean_fiat', 'max_flowiat', 'mean_biat', 'std_flowiat', 'min_flowiat', 'duration', 'class1']


In [178]:
import numpy as np
import pandas as pd

def removeNanVlaues(df):
    df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace=True)
    return df
train = df.values
train= df.dropna()
train.shape

(9564, 14)

In [179]:
#y_train = train[:,len(df.columns)-1]
y_train = train.iloc[:,13:14].values#,len(df.columns)-1]
x_train = train.iloc[:,0:13].values#,len(df.columns)-1]
print(x_train[0:2],y_train[0:2])
y_train.shape

[[3.51491420e+07 3.51884370e+07 5.57867233e+05 4.74297571e+05
  1.41164010e+03 2.55901044e+05 3.91629300e+00 2.50225706e+06
  3.51376500e+07 2.69357556e+06 1.83954785e+06 4.00000000e+00
  1.17202678e+08]
 [3.50228770e+07 3.52089500e+07 5.41003561e+05 5.10804295e+05
  1.73110465e+03 2.62234950e+05 3.82173700e+00 2.58205186e+06
  3.50228770e+07 2.63444747e+06 1.84945228e+06 4.00000000e+00
  1.19579137e+08]] [['NONTOR']
 ['NONTOR']]


(9564, 1)

In [180]:
lb = preprocessing.LabelBinarizer()
y_train = lb.fit_transform(y_train)
print(len(y_train))
print(y_train.shape)
print(y_train)

y_binary = to_categorical(y_train)
print(y_binary)
print(y_binary.shape)

9564
(9564, 1)
[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
(9564, 2)


In [181]:
#trainX = train[:, :len(df.columns)-1]
#print(trainX.shape)
#trainX

In [182]:
trainX = x_train.reshape(x_train.shape[0],len(fields)-1,1).astype( 'float32' )
print(trainX.shape)
trainX


(9564, 13, 1)


array([[[3.5149144e+07],
        [3.5188436e+07],
        [5.5786725e+05],
        ...,
        [1.8395479e+06],
        [4.0000000e+00],
        [1.1720268e+08]],

       [[3.5022876e+07],
        [3.5208952e+07],
        [5.4100356e+05],
        ...,
        [1.8494522e+06],
        [4.0000000e+00],
        [1.1957914e+08]],

       [[2.5978548e+07],
        [2.6116332e+07],
        [7.5039625e+05],
        ...,
        [2.3134988e+06],
        [5.0000000e+00],
        [6.5341088e+07]],

       ...,

       [[1.3000000e+01],
        [7.2000000e+01],
        [7.5980000e+04],
        ...,
        [2.3026570e+04],
        [8.0000000e+00],
        [1.1993364e+08]],

       [[1.4000000e+01],
        [4.4000000e+01],
        [7.6807000e+04],
        ...,
        [2.3030076e+04],
        [9.0000000e+00],
        [1.1999071e+08]],

       [[1.4700000e+02],
        [7.6000000e+01],
        [8.9778000e+05],
        ...,
        [3.6154887e+04],
        [1.0000000e+01],
        [4.6755396e+07]]

In [183]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.33, shuffle= True)
print(len(x_train))
print(x_train.shape)
print(len(y_train))
print(y_train.shape)

6407
(6407, 13)
6407
(6407, 1)


In [184]:
from sklearn.preprocessing import StandardScaler

def dataPreProcessing(x):
    sc = StandardScaler()
    x = sc.fit_transform(x)
    print(x.shape)
    return x

x_train = dataPreProcessing(x_train)

(6407, 13)


In [186]:
    model = Sequential()
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(len(fields)-1,1),padding='same'))

    #model.add(Conv1D(filters=128, kernel_size=3, activation='relu', kernel_initializer='random_normal',
                     #input_shape=(input_dim,1),
                     #padding='same'))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu', kernel_initializer='random_normal', padding='same'))
    #model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))

    model.add(Conv1D(filters=256, kernel_size=3, activation='relu', kernel_initializer='random_normal', padding='same'))
    model.add(Conv1D(filters=256, kernel_size=3, activation='relu', kernel_initializer='random_normal', padding='same'))
    #model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))

    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='sigmoid'))
    
    LEARN_RATE = 1.0e-2
    model.compile(loss='binary_crossentropy', # Better loss function for neural networks
              optimizer=Adam(learning_rate=LEARN_RATE), # Adam optimizer with 1.0e-4 learning rate
                     metrics = ['accuracy']) # Metrics to be evaluated by the model
    (model.summary())

C:\Users\tm120a\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_24"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_90 (Conv1D)              │ (None, 13, 128)        │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_91 (Conv1D)              │ (None, 13, 128)        │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_44 (MaxPooling1D) │ (None, 6, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_92 (Conv1D)              │ (None, 6, 256)         │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_93 (Conv1D)              │ (None, 6, 256)         │       196,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_45 (MaxPooling1D) │ (None, 3, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_22 (Flatten)            │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_65 (Dense)                │ (None, 128)            │        98,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_66 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_67 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 451,969 (1.72 MB)

 Trainable params: 451,969 (1.72 MB)

 Non-trainable params: 0 (0.00 B)

#### !!!!!!!!! Feature Selection Method: INFO GAIN!!!!!!!!!!! #####

## Training results: TOR non-TOR for 60 sec "TimeBasedFeatures-30s-TOR-NonTOR-85.arff"

In [149]:
history1 = model.fit(x_train, y_train,
                     epochs=20,
                    # validation_data=(x_valid,y_valid),
                     shuffle=True,
                   #  callbacks=[sgdr],
                     batch_size= 64)

Epoch 1/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 0.9431 - loss: 0.1365
Epoch 2/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.9997 - loss: 0.0040
Epoch 3/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9996 - loss: 0.0049
Epoch 4/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.9988 - loss: 0.0178
Epoch 5/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.9998 - loss: 0.0219
Epoch 6/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.9993 - loss: 0.0051
Epoch 7/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.9999 - loss: 0.0019
Epoch 8/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.9986 - loss: 0.0217
Epoch 9/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.9825 - loss: 0.2214
Epoch 10/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.9991 - loss: 0.0083
Epoch 11/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.9997 - loss: 0.0034
Epoch 12/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 

## Training results: TOR non-TOR for 30 sec "TimeBasedFeatures-30s-TOR-NonTOR-85.arff"

In [162]:
history1 = model.fit(x_train, y_train,
                     epochs=20,
                    # validation_data=(x_valid,y_valid),
                     shuffle=True,
                   #  callbacks=[sgdr],
                     batch_size= 64)

Epoch 1/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 9s 26ms/step - accuracy: 0.8812 - loss: 0.2858
Epoch 2/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.9440 - loss: 0.1597
Epoch 3/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.9623 - loss: 0.1165
Epoch 4/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.9458 - loss: 0.2324
Epoch 5/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.9646 - loss: 0.0966
Epoch 6/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.9686 - loss: 0.0930
Epoch 7/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.9892 - loss: 0.0373
Epoch 8/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.9604 - loss: 0.1196
Epoch 9/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.9913 - loss: 0.1102
Epoch 10/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - accuracy: 0.9415 - loss: 0.1282
Epoch 11/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.9860 - loss: 0.0472
Epoch 12/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 

## Training results: TOR non-TOR for 15 sec "TimeBasedFeatures-30s-TOR-NonTOR-85.arff"

In [174]:
history1 = model.fit(x_train, y_train,
                     epochs=20,
                    # validation_data=(x_valid,y_valid),
                     shuffle=True,
                   #  callbacks=[sgdr],
                     batch_size= 64)

Epoch 1/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.8509 - loss: 0.3537
Epoch 2/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.9922 - loss: 0.0349
Epoch 3/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.9842 - loss: 0.0696
Epoch 4/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.9936 - loss: 0.0344
Epoch 5/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9907 - loss: 0.0400
Epoch 6/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.9976 - loss: 0.0152
Epoch 7/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.9952 - loss: 0.0241
Epoch 8/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.9974 - loss: 0.0173
Epoch 9/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9962 - loss: 0.0224
Epoch 10/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.9947 - loss: 0.0723
Epoch 11/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 0.9964 - loss: 0.0200
Epoch 12/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 

## Training results: TOR non-TOR for 120 sec "TimeBasedFeatures-30s-TOR-NonTOR-85.arff"

In [187]:
history1 = model.fit(x_train, y_train,
                     epochs=20,
                    # validation_data=(x_valid,y_valid),
                     shuffle=True,
                   #  callbacks=[sgdr],
                     batch_size= 64)

Epoch 1/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - accuracy: 0.9601 - loss: 0.2359
Epoch 2/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 0.9721 - loss: 0.0667
Epoch 3/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.9793 - loss: 0.0682
Epoch 4/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.9804 - loss: 0.0533
Epoch 5/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.9799 - loss: 0.0641
Epoch 6/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.9832 - loss: 0.0480
Epoch 7/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.9768 - loss: 0.0670
Epoch 8/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.9785 - loss: 0.0490
Epoch 9/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.9837 - loss: 0.0362
Epoch 10/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 0.9802 - loss: 0.0480
Epoch 11/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.9833 - loss: 0.0731
Epoch 12/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 

#### !!!!!!!!! Feature Selection Method: CORRELATION!!!!!!!!!!! #####

In [20]:
import scipy.io.arff
import pandas as pd

# Load the ARFF file
########30s###TOR non TOR###data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TORnTOR30secTraining.arff')#TORnTOR30secTraining
data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-60s-TOR-NonTOR-85.arff')#TORnTOR30secTraining
##data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-30s-TORNonTOR-85.arff')#TORnTOR30secTraining
##data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-15s-TOR-NonTOR-85.arff')#TORnTOR30secTraining
##data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-120s-TOR-NonTOR-85.arff')#TORnTOR30secTraining
##data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-60s-TOR-NonTOR-85.arff')#TORnTOR30secTraining




# Convert to pandas DataFrame for easier manipulation
df = pd.DataFrame(data)

# Convert any byte-like strings to regular strings (if needed)
df = df.apply(lambda x: x.str.decode('utf-8') if x.dtype == 'O' else x)

# Define the columns you're interested in
#fields = ['min_biat','min_fiat','max_biat','max_fiat','flowBytesPerSecond','mean_flowiat',
 #         'flowPktsPerSecond','mean_fiat','max_flowiat','mean_biat','std_flowiat','min_flowiat','duration','class1']
fields = ['min_fiat','min_biat','mean_flowiat','duration','std_flowiat','max_biat',
             #      'max_idle','max_flowiat','mean_idle','mean_biat','min_idle','max_fiat','mean_fiat','max_active','class1']


# Select only the specified columns
df= df[fields]

# Show the first few rows of the selected columns
df.head()


<>:6: SyntaxWarning: invalid escape sequence '\T'
<>:6: SyntaxWarning: invalid escape sequence '\T'
C:\Users\tm120a\AppData\Local\Temp\ipykernel_19916\3472741151.py:6: SyntaxWarning: invalid escape sequence '\T'
  data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-60s-TOR-NonTOR-85.arff')#TORnTOR30secTraining
C:\Users\tm120a\AppData\Local\Temp\ipykernel_19916\3472741151.py:6: SyntaxWarning: invalid escape sequence '\T'
  data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-60s-TOR-NonTOR-85.arff')#TORnTOR30secTraining


KeyError: "None of [Index(['Flow Bytes/s', 'Fwd IAT Mean', 'Flow IAT Mean', 'Flow Packets/s',\n       'Fwd IAT Max', 'Flow IAT Max', 'Bwd IAT Max', 'Flow IAT Std',\n       'Bwd IAT Mean', 'Bwd IAT Min', 'Fwd IAT Std', 'Bwd IAT Std',\n       'Fwd IAT Min', 'Flow IAT Min', 'label'],\n      dtype='object')] are in the [columns]"

In [7]:
import numpy as np
import pandas as pd

def removeNanVlaues(df):
    df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace=True)
    return df
train = df.values
train= df.dropna()
train.shape

y_train = train.iloc[:,14:15].values#,len(df.columns)-1]
x_train = train.iloc[:,0:14].values#,len(df.columns)-1]
print(x_train[0:2],y_train[0:2])
y_train.shape

lb = preprocessing.LabelBinarizer()
y_train = lb.fit_transform(y_train)
print(len(y_train))
print(y_train.shape)
print(y_train)

y_binary = to_categorical(y_train)
print(y_binary)
print(y_binary.shape)

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.33, shuffle= True)
print(len(x_train))
print(x_train.shape)
print(len(y_train))
print(y_train.shape)

from sklearn.preprocessing import StandardScaler

def dataPreProcessing(x):
    sc = StandardScaler()
    x = sc.fit_transform(x)
    print(x.shape)
    return x

x_train = dataPreProcessing(x_train)

    

[[-1. -1. 76. 76.  0. -1. -1. 76.  0.  0. -1. -1.  0. -1.]
 [-1. -1. 54. 54.  0. -1. -1. 54.  0.  0. -1. -1.  0. -1.]] [['NONTOR']
 ['NONTOR']]
13964
(13964, 1)
[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
(13964, 2)
9355
(9355, 14)
9355
(9355, 1)
(9355, 14)


In [8]:
    model = Sequential()
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(len(fields)-1,1),padding='same'))

    #model.add(Conv1D(filters=128, kernel_size=3, activation='relu', kernel_initializer='random_normal',
                     #input_shape=(input_dim,1),
                     #padding='same'))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu', kernel_initializer='random_normal', padding='same'))
    #model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))

    model.add(Conv1D(filters=256, kernel_size=3, activation='relu', kernel_initializer='random_normal', padding='same'))
    model.add(Conv1D(filters=256, kernel_size=3, activation='relu', kernel_initializer='random_normal', padding='same'))
    #model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))

    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='sigmoid'))
    
    LEARN_RATE = 1.0e-2
    model.compile(loss='binary_crossentropy', # Better loss function for neural networks
              optimizer=Adam(learning_rate=LEARN_RATE), # Adam optimizer with 1.0e-4 learning rate
                     metrics = ['accuracy']) # Metrics to be evaluated by the model
    (model.summary())

C:\Users\tm120a\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 14, 128)        │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 14, 128)        │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 7, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 7, 256)         │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 7, 256)         │       196,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 3, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        98,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 451,969 (1.72 MB)

 Trainable params: 451,969 (1.72 MB)

 Non-trainable params: 0 (0.00 B)

## TOR non-TOR 60 sec Correlation-based FSM ##

In [214]:
history1 = model.fit(x_train, y_train,
                     epochs=20,
                    # validation_data=(x_valid,y_valid),
                     shuffle=True,
                   #  callbacks=[sgdr],
                     batch_size= 64)

Epoch 1/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.9996 - loss: 0.0041
Epoch 2/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.9995 - loss: 0.0039
Epoch 3/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - accuracy: 0.9994 - loss: 0.0081
Epoch 4/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.9992 - loss: 0.0084
Epoch 5/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - accuracy: 0.9997 - loss: 0.0034
Epoch 6/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.9993 - loss: 0.0052
Epoch 7/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - accuracy: 0.9996 - loss: 0.0037
Epoch 8/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - accuracy: 0.9994 - loss: 0.0055
Epoch 9/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - accuracy: 0.9992 - loss: 0.0065
Epoch 10/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - accuracy: 0.9997 - loss: 0.0029
Epoch 11/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.9996 - loss: 0.0035
Epoch 12/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 

## TOR non-TOR 30 sec Correlation-based FSM ##

In [204]:
history1 = model.fit(x_train, y_train,
                     epochs=20,
                    # validation_data=(x_valid,y_valid),
                     shuffle=True,
                   #  callbacks=[sgdr],
                     batch_size= 64)

Epoch 1/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.8487 - loss: 32.4436
Epoch 2/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.8949 - loss: 0.1753
Epoch 3/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - accuracy: 0.9375 - loss: 0.1666
Epoch 4/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - accuracy: 0.9412 - loss: 0.1415
Epoch 5/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.9418 - loss: 0.1354
Epoch 6/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - accuracy: 0.9527 - loss: 0.1227
Epoch 7/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.9456 - loss: 0.1224
Epoch 8/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.9507 - loss: 0.1600
Epoch 9/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - accuracy: 0.9410 - loss: 0.1263
Epoch 10/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.9412 - loss: 0.1390
Epoch 11/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.9517 - loss: 0.1095
Epoch 12/20
147/147 ━━━━━━━━━━━━━━━━━━━━

## TOR non-TOR 120 sec Correlation-based FSM ##

In [208]:
history1 = model.fit(x_train, y_train,
                     epochs=20,
                    # validation_data=(x_valid,y_valid),
                     shuffle=True,
                   #  callbacks=[sgdr],
                     batch_size= 64)

Epoch 1/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 7s 25ms/step - accuracy: 0.9398 - loss: 0.2152
Epoch 2/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.9738 - loss: 0.0719
Epoch 3/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.9688 - loss: 0.1040
Epoch 4/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.9685 - loss: 0.0712
Epoch 5/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.9789 - loss: 0.0581
Epoch 6/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.9738 - loss: 0.0586
Epoch 7/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.9761 - loss: 0.0810
Epoch 8/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.9808 - loss: 0.0630
Epoch 9/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.9650 - loss: 0.1336
Epoch 10/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 0.9577 - loss: 0.0816
Epoch 11/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.9570 - loss: 0.0752
Epoch 12/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 


#### DNN Results for Infogain #####


In [152]:
import scipy.io.arff
import pandas as pd



#fields = ['Flow Bytes/s', 'Fwd IAT Mean','Flow IAT Mean','Flow Packets/s','Fwd IAT Max','Flow IAT Max',
       #   'Bwd IAT Max','Flow IAT Std','Bwd IAT Mean','Bwd IAT Min','Fwd IAT Std','Bwd IAT Std','Fwd IAT Min',
        #  'Flow IAT Min','label']
# Load the ARFF file
########30s###TOR non TOR###data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TORnTOR30secTraining.arff')#TORnTOR30secTraining
#data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-60s-TOR-NonTOR-85.arff')#TORnTOR30secTraining
##data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-30s-TORNonTOR-85.arff')#TORnTOR30secTraining
##data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-15s-TOR-NonTOR-85.arff')#TORnTOR30secTraining
##data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-120s-TOR-NonTOR-85.arff')#TORnTOR30secTraining
data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-15s-Layer2.arff')#TORnTOR30secTraining
#data = pd.read_csv('DatasetTORnTOR\Scenario-B-merged_5s.csv',skipinitialspace=True, usecols=fields)


# Convert to pandas DataFrame for easier manipulation
df = pd.DataFrame(data)

# Convert any byte-like strings to regular strings (if needed)
df = df.apply(lambda x: x.str.decode('utf-8') if x.dtype == 'O' else x)

# Define the columns you're interested in
#fields = ['min_biat','min_fiat','max_biat','max_fiat','flowBytesPerSecond','mean_flowiat',
 #         'flowPktsPerSecond','mean_fiat','max_flowiat','mean_biat','std_flowiat','min_flowiat','duration','class1']
fields = ['min_fiat','min_biat','mean_flowiat','duration','std_flowiat','max_biat',
                  'max_idle','max_flowiat','mean_idle','mean_biat','min_idle','max_fiat','mean_fiat','max_active','class1']

# Select only the specified columns
df= df[fields]

# Show the first few rows of the selected columns
df.head()


<>:15: SyntaxWarning: invalid escape sequence '\T'
<>:15: SyntaxWarning: invalid escape sequence '\T'
C:\Users\tm120a\AppData\Local\Temp\ipykernel_19916\237500454.py:15: SyntaxWarning: invalid escape sequence '\T'
  data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-15s-Layer2.arff')#TORnTOR30secTraining


,min_fiat,min_biat,mean_flowiat,duration,std_flowiat,max_biat,max_idle,max_flowiat,mean_idle,mean_biat,min_idle,max_fiat,mean_fiat,max_active,class1
0,60.0,52.0,240588.372093,10345300.0,1.022685e+06,5870638.0,5870638.0,5870638.0,4653308.5,470235.181818,3435979.0,5871778.0,517262.850000,5870764.0,CHAT
1,7.0,0.0,3658.360548,14966353.0,2.134758e+04,635242.0,-1.0,635042.0,0.0,5615.779737,-1.0,635321.0,10502.596491,-1.0,VIDEO-STREAMING
2,20303.0,19353.0,54573.400000,272867.0,9.018174e+04,253467.0,-1.0,213324.0,0.0,136410.000000,-1.0,213324.0,116813.500000,-1.0,CHAT
3,0.0,0.0,3119.673669,14999391.0,9.995782e+03,146556.0,-1.0,146556.0,0.0,4758.690038,-1.0,149034.0,9062.771601,-1.0,VIDEO-STREAMING
4,116.0,492.0,422976.294118,7190597.0,1.582350e+06,6594711.0,6555090.0,6555090.0,6555090.0,898824.625000,6555090.0,6555090.0,898743.375000,7051063.0,CHAT


In [153]:
import numpy as np
import pandas as pd

def removeNanVlaues(df):
    df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace=True)
    return df
train = df.values
train= df.dropna()
train.shape



y_train = train.iloc[:,14:15].values#,len(df.columns)-1]
x_train = train.iloc[:,0:14].values#,len(df.columns)-1]
print(x_train[0:2],y_train[0:2])
y_train.shape

lb = preprocessing.LabelBinarizer()
y_train = lb.fit_transform(y_train)
print(len(y_train))
print(y_train.shape)
print(y_train)

y_binary = to_categorical(y_train)
print(y_binary)
print(y_binary.shape)

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.33, shuffle= False)
print(len(x_train))
print(x_train.shape)
print(len(y_train))
print('y train shape',y_train.shape)





from sklearn.preprocessing import StandardScaler

def dataPreProcessing(x):
    sc = StandardScaler()
    x = sc.fit_transform(x)
    #print(x.shape)
    return x

x_train = dataPreProcessing(x_train)

    

[[ 6.00000000e+01  5.20000000e+01  2.40588372e+05  1.03453000e+07
   1.02268473e+06  5.87063800e+06  5.87063800e+06  5.87063800e+06
   4.65330850e+06  4.70235182e+05  3.43597900e+06  5.87177800e+06
   5.17262850e+05  5.87076400e+06]
 [ 7.00000000e+00  0.00000000e+00  3.65836055e+03  1.49663530e+07
   2.13475751e+04  6.35242000e+05 -1.00000000e+00  6.35042000e+05
   0.00000000e+00  5.61577974e+03 -1.00000000e+00  6.35321000e+05
   1.05025965e+04 -1.00000000e+00]] [['CHAT']
 ['VIDEO-STREAMING']]
3360
(3360, 8)
[[0 0 1 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]
[[[1. 0.]
  [1. 0.]
  [0. 1.]
  ...
  [1. 0.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]
  ...
  [1. 0.]
  [0. 1.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [0. 1.]
  ...
  [1. 0.]
  [1. 0.]
  [1. 0.]]

 ...

 [[1. 0.]
  [1. 0.]
  [1. 0.]
  ...
  [1. 0.]
  [1. 0.]
  [0. 1.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]
  ...
  [1. 0.]
  [1. 0.]
  [0. 1.]]

 [[1. 0.]
  [1. 0.]


In [150]:
    model = Sequential()
    model.add(Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(len(fields)-1,1),padding='same'))

    #model.add(Conv1D(filters=128, kernel_size=3, activation='relu', kernel_initializer='random_normal',
                     #input_shape=(input_dim,1),
                     #padding='same'))
    model.add(Conv1D(filters=256, kernel_size=3, activation='relu', kernel_initializer='random_normal', padding='same'))
    #model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))

    model.add(Conv1D(filters=512, kernel_size=3, activation='relu', kernel_initializer='random_normal', padding='same'))
    model.add(Conv1D(filters=512, kernel_size=3, activation='relu', kernel_initializer='random_normal', padding='same'))
    #model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))

    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='sigmoid'))
    
    LEARN_RATE = 1.0e-2
    model.compile(loss='categorical_crossentropy', # Better loss function for neural networks
              optimizer=Adam(learning_rate=LEARN_RATE), # Adam optimizer with 1.0e-4 learning rate
                     metrics = ['accuracy']) # Metrics to be evaluated by the model
    (model.summary())

Model: "sequential_48"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_56 (Conv1D)              │ (None, 13, 256)        │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_57 (Conv1D)              │ (None, 13, 256)        │       196,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_28 (MaxPooling1D) │ (None, 6, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_58 (Conv1D)              │ (None, 6, 512)         │       393,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_59 (Conv1D)              │ (None, 6, 512)         │       786,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_29 (MaxPooling1D) │ (None, 3, 512)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_48 (Flatten)            │ (None, 1536)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_192 (Dense)               │ (None, 128)            │       196,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_193 (Dense)               │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_194 (Dense)               │ (None, 8)              │           520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,584,072 (6.04 MB)

 Trainable params: 1,584,072 (6.04 MB)

 Non-trainable params: 0 (0.00 B)

In [154]:
    classifier = Sequential()
    #First Hidden Layer
    classifier.add(Dense(350, activation='relu', kernel_initializer='random_normal', input_shape=(len(fields)-1,1)))
    #Second  Hidden Layer
    classifier.add(Dense(350, activation='relu', kernel_initializer='random_normal'))
    #Third Hidden Layer
    #classifier.add(Dense(150, activation='relu'))#, kernel_initializer='random_normal'))


    #Output Layer
    classifier.add(Flatten())
    classifier.add(Dense(y_train.shape[1], activation='softmax'))#, kernel_initializer='random_normal'))
    #Compile the model
    #classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])
    classifier.compile(loss='categorical_crossentropy', # Better loss function for neural networks
              optimizer=Adam(learning_rate=0.001), # Adam optimizer with 1.0e-4 learning rate
                     metrics = ['accuracy'])

classifier.summary()

C:\Users\tm120a\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_49"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_195 (Dense)               │ (None, 14, 350)        │           700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_196 (Dense)               │ (None, 14, 350)        │       122,850 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_49 (Flatten)            │ (None, 4900)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_197 (Dense)               │ (None, 8)              │        39,208 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 162,758 (635.77 KB)

 Trainable params: 162,758 (635.77 KB)

 Non-trainable params: 0 (0.00 B)

In [155]:
### for DNN use classifier.fit ######
history1 = classifier.fit(x_train, y_train,
                     epochs=20,
                    # validation_data=(x_valid,y_valid),
                     shuffle=True,
                   #  callbacks=[sgdr],
                     batch_size= 64)

Epoch 1/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.4072 - loss: 1.7619
Epoch 2/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.4703 - loss: 1.3531
Epoch 3/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5023 - loss: 1.2918
Epoch 4/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5018 - loss: 1.2669
Epoch 5/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.5189 - loss: 1.2060
Epoch 6/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.5545 - loss: 1.1994
Epoch 7/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5637 - loss: 1.0940
Epoch 8/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.5935 - loss: 1.0892
Epoch 9/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5980 - loss: 1.0594
Epoch 10/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.6048 - loss: 1.0621
Epoch 11/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.6179 - loss: 0.9833
Epoch 12/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy:

In [126]:
    model = Sequential()
    model.add(Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(len(fields)-1,1),padding='same'))

    #model.add(Conv1D(filters=128, kernel_size=3, activation='relu', kernel_initializer='random_normal',
                     #input_shape=(input_dim,1),
                     #padding='same'))
    model.add(Conv1D(filters=256, kernel_size=3, activation='relu', kernel_initializer='random_normal', padding='same'))
    #model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))

    model.add(Conv1D(filters=512, kernel_size=3, activation='relu', kernel_initializer='random_normal', padding='same'))
    model.add(Conv1D(filters=512, kernel_size=3, activation='relu', kernel_initializer='random_normal', padding='same'))
    #model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))

    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='sigmoid'))
    
    LEARN_RATE = 1.0e-2
    model.compile(loss='categorical_crossentropy', # Better loss function for neural networks
              optimizer=Adam(learning_rate=LEARN_RATE), # Adam optimizer with 1.0e-4 learning rate
                     metrics = ['accuracy']) # Metrics to be evaluated by the model
    (model.summary())

Model: "sequential_38"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_44 (Conv1D)              │ (None, 14, 256)        │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_45 (Conv1D)              │ (None, 14, 256)        │       196,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_22 (MaxPooling1D) │ (None, 7, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_46 (Conv1D)              │ (None, 7, 512)         │       393,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_47 (Conv1D)              │ (None, 7, 512)         │       786,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_23 (MaxPooling1D) │ (None, 3, 512)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_38 (Flatten)            │ (None, 1536)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_155 (Dense)               │ (None, 128)            │       196,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_156 (Dense)               │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_157 (Dense)               │ (None, 8)              │           520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,584,072 (6.04 MB)

 Trainable params: 1,584,072 (6.04 MB)

 Non-trainable params: 0 (0.00 B)

In [127]:
### for CNN use model.fit ######
history1 = model.fit(x_train, y_train,
                     epochs=100,
                    # validation_data=(x_valid,y_valid),
                     shuffle=True,
                   #  callbacks=[sgdr],
                     batch_size= 64)

Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 64ms/step - accuracy: 0.2419 - loss: 2.2515
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 0.4768 - loss: 1.4421
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 0.4771 - loss: 1.4223
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - accuracy: 0.4728 - loss: 1.4428
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - accuracy: 0.4709 - loss: 1.3361
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 0.4712 - loss: 1.3502
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - accuracy: 0.5152 - loss: 1.2995
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - accuracy: 0.5130 - loss: 1.3059
Epoch 9/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accuracy: 0.4746 - loss: 1.3866
Epoch 10/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - accuracy: 0.5008 - loss: 1.2975
Epoch 11/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 0.5438 - loss: 1.1631
Epoch 12/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step

KeyboardInterrupt: 

### DNN results 60sec infogain######

In [240]:
import scipy.io.arff
import pandas as pd

# Load the ARFF file
########30s###TOR non TOR###data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TORnTOR30secTraining.arff')#TORnTOR30secTraining
data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-60s-TOR-NonTOR-85.arff')#TORnTOR30secTraining
##data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-30s-TORNonTOR-85.arff')#TORnTOR30secTraining
##data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-15s-TOR-NonTOR-85.arff')#TORnTOR30secTraining
##data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-120s-TOR-NonTOR-85.arff')#TORnTOR30secTraining
##data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-60s-TOR-NonTOR-85.arff')#TORnTOR30secTraining




# Convert to pandas DataFrame for easier manipulation
df = pd.DataFrame(data)

# Convert any byte-like strings to regular strings (if needed)
df = df.apply(lambda x: x.str.decode('utf-8') if x.dtype == 'O' else x)

# Define the columns you're interested in for Correlato
fields = ['min_biat','min_fiat','max_biat','max_fiat','flowBytesPerSecond','mean_flowiat',
          'flowPktsPerSecond','mean_fiat','max_flowiat','mean_biat','std_flowiat','min_flowiat','duration','class1']

######## use for INFOGAIN ###################
#fields = ['min_fiat','min_biat','mean_flowiat','duration','std_flowiat','max_biat',
 #                  'max_idle','max_flowiat','mean_idle','mean_biat','min_idle','max_fiat','mean_fiat','max_active','class1']##infogain

# Select only the specified columns
df= df[fields]

# Show the first few rows of the selected columns
df.head()


<>:6: SyntaxWarning: invalid escape sequence '\T'
<>:6: SyntaxWarning: invalid escape sequence '\T'
C:\Users\tm120a\AppData\Local\Temp\ipykernel_35420\1029684786.py:6: SyntaxWarning: invalid escape sequence '\T'
  data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-60s-TOR-NonTOR-85.arff')#TORnTOR30secTraining


,min_biat,min_fiat,max_biat,max_fiat,flowBytesPerSecond,mean_flowiat,flowPktsPerSecond,mean_fiat,max_flowiat,mean_biat,std_flowiat,min_flowiat,duration,class1
0,-1.0,-1.0,-1.0,-1.0,1.052632e+06,76.0,26315.789474,0.0,76.0,0.0,0.0,76.0,76.0,NONTOR
1,-1.0,-1.0,-1.0,-1.0,1.481481e+06,54.0,37037.037037,0.0,54.0,0.0,0.0,54.0,54.0,NONTOR
2,-1.0,-1.0,-1.0,-1.0,1.066667e+06,75.0,26666.666667,0.0,75.0,0.0,0.0,75.0,75.0,NONTOR
3,-1.0,14512571.0,-1.0,14512571.0,8.268693e+00,7256285.5,0.206717,14500000.0,14512502.0,0.0,10300000.0,69.0,14512571.0,NONTOR
4,-1.0,-1.0,-1.0,-1.0,1.951220e+06,41.0,48780.487805,0.0,41.0,0.0,0.0,41.0,41.0,NONTOR


In [343]:
import numpy as np
import pandas as pd

def removeNanVlaues(df):
    df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace=True)
    return df
train = df.values
train= df.dropna()
train.shape

y_train = train.iloc[:,13:14].values#,len(df.columns)-1]
x_train = train.iloc[:,0:13].values#,len(df.columns)-1]
print(x_train[0:2],y_train[0:2])
y_train.shape

lb = preprocessing.LabelBinarizer()
y_train = lb.fit_transform(y_train)
print(len(y_train))
print(y_train.shape)
print(y_train)

y_binary = to_categorical(y_train)
print(y_binary)
print(y_binary.shape)

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.33, shuffle= True)
print(len(x_train))
print(x_train.shape)
print(len(y_train))
print(y_train.shape)

from sklearn.preprocessing import StandardScaler

def dataPreProcessing(x):
    sc = StandardScaler()
    x = sc.fit_transform(x)
    print(x.shape)
    return x

x_train = dataPreProcessing(x_train)

    

[[-1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
   1.05263158e+06  7.60000000e+01  2.63157895e+04  0.00000000e+00
   7.60000000e+01  0.00000000e+00  0.00000000e+00  7.60000000e+01
   7.60000000e+01]
 [-1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
   1.48148148e+06  5.40000000e+01  3.70370370e+04  0.00000000e+00
   5.40000000e+01  0.00000000e+00  0.00000000e+00  5.40000000e+01
   5.40000000e+01]] [['NONTOR']
 ['NONTOR']]
13964
(13964, 1)
[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
(13964, 2)
9355
(9355, 13)
9355
(9355, 1)
(9355, 13)


In [356]:
    classifier = Sequential()
    #First Hidden Layer
    classifier.add(Dense(50, activation='relu', kernel_initializer='random_normal', input_shape=(len(fields)-1,1)))
    #Second  Hidden Layer
    classifier.add(Dense(50, activation='relu', kernel_initializer='random_normal'))
    #Third Hidden Layer
    classifier.add(Dense(50, activation='relu', kernel_initializer='random_normal'))
    #Output Layer
    classifier.add(Flatten())
    classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
   
    #Compile the model
    
    #classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])
    classifier.compile(loss='binary_crossentropy', # Better loss function for neural networks
              optimizer=Adam(learning_rate=LEARN_RATE), # Adam optimizer with 1.0e-4 learning rate
                     metrics = ['accuracy'])
    classifier.summary()

Model: "sequential_85"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_295 (Dense)               │ (None, 14, 50)         │           100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_296 (Dense)               │ (None, 14, 50)         │         2,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_297 (Dense)               │ (None, 14, 50)         │         2,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_31 (Flatten)            │ (None, 700)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_298 (Dense)               │ (None, 1)              │           701 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,901 (23.05 KB)

 Trainable params: 5,901 (23.05 KB)

 Non-trainable params: 0 (0.00 B)

In [357]:
history1 = classifier.fit(x_train, y_train,
                     epochs=20,
                    # validation_data=(x_valid,y_valid),
                     shuffle=True,
                   #  callbacks=[sgdr],
                     batch_size= 64)

Epoch 1/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9540 - loss: 0.1441
Epoch 2/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9985 - loss: 0.0055
Epoch 3/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9998 - loss: 0.0028
Epoch 4/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9998 - loss: 0.0023
Epoch 5/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9999 - loss: 9.9722e-04
Epoch 6/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9997 - loss: 0.0030
Epoch 7/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9999 - loss: 0.0012
Epoch 8/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9996 - loss: 0.0057
Epoch 9/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9996 - loss: 0.0039
Epoch 10/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9999 - loss: 0.0014
Epoch 11/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9999 - loss: 0.0012
Epoch 12/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/

### TESTING FOR APPLICATION SERVICES ~################

In [131]:
import scipy.io.arff
import pandas as pd



#fields = ['Flow Bytes/s', 'Fwd IAT Mean','Flow IAT Mean','Flow Packets/s','Fwd IAT Max','Flow IAT Max',
       #   'Bwd IAT Max','Flow IAT Std','Bwd IAT Mean','Bwd IAT Min','Fwd IAT Std','Bwd IAT Std','Fwd IAT Min',
        #  'Flow IAT Min','label']
# Load the ARFF file
########30s###TOR non TOR###data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TORnTOR30secTraining.arff')#TORnTOR30secTraining
#data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-60s-TOR-NonTOR-85.arff')#TORnTOR30secTraining
##data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-30s-TORNonTOR-85.arff')#TORnTOR30secTraining
##data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-15s-TOR-NonTOR-85.arff')#TORnTOR30secTraining
##data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-120s-TOR-NonTOR-85.arff')#TORnTOR30secTraining
data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-15s-Layer2.arff')#TORnTOR30secTraining
#data = pd.read_csv('DatasetTORnTOR\Scenario-B-merged_5s.csv',skipinitialspace=True, usecols=fields)


# Convert to pandas DataFrame for easier manipulation
df = pd.DataFrame(data)

# Convert any byte-like strings to regular strings (if needed)
df = df.apply(lambda x: x.str.decode('utf-8') if x.dtype == 'O' else x)

# Define the columns you're interested in
fields = ['min_biat','min_fiat','max_biat','max_fiat','flowBytesPerSecond','mean_flowiat',
          'flowPktsPerSecond','mean_fiat','max_flowiat','mean_biat','std_flowiat','min_flowiat','duration','class1']
#fields = ['min_fiat','min_biat','mean_flowiat','duration','std_flowiat','max_biat',
 #                 'max_idle','max_flowiat','mean_idle','mean_biat','min_idle','max_fiat','mean_fiat','max_active','class1']

# Select only the specified columns
df= df[fields]

# Show the first few rows of the selected columns
df.head()


<>:15: SyntaxWarning: invalid escape sequence '\T'
<>:15: SyntaxWarning: invalid escape sequence '\T'
C:\Users\tm120a\AppData\Local\Temp\ipykernel_19916\2465855891.py:15: SyntaxWarning: invalid escape sequence '\T'
  data, meta = scipy.io.arff.loadarff('DatasetTORnTOR\TimeBasedFeatures-15s-Layer2.arff')#TORnTOR30secTraining


,min_biat,min_fiat,max_biat,max_fiat,flowBytesPerSecond,mean_flowiat,flowPktsPerSecond,mean_fiat,max_flowiat,mean_biat,std_flowiat,min_flowiat,duration,class1
0,52.0,60.0,5870638.0,5871778.0,2260.446773,240588.372093,4.253139,517262.850000,5870638.0,470235.181818,1.022685e+06,13.0,10345300.0,CHAT
1,0.0,7.0,635242.0,635321.0,227450.334761,3658.360548,273.413302,10502.596491,635042.0,5615.779737,2.134758e+04,0.0,14966353.0,VIDEO-STREAMING
2,19353.0,20303.0,253467.0,213324.0,8733.192361,54573.400000,21.988734,116813.500000,213324.0,136410.000000,9.018174e+04,47.0,272867.0,CHAT
3,0.0,0.0,146556.0,149034.0,277900.949445,3119.673669,320.613017,9062.771601,146556.0,4758.690038,9.995782e+03,0.0,14999391.0,VIDEO-STREAMING
4,492.0,116.0,6594711.0,6555090.0,1137.179569,422976.294118,2.503269,898743.375000,6555090.0,898824.625000,1.582350e+06,12.0,7190597.0,CHAT


In [135]:
import numpy as np
import pandas as pd

def removeNanVlaues(df):
    df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace=True)
    return df
train = df.values
train= df.dropna()
train.shape

y_train = train.iloc[:,13:14].values#,len(df.columns)-1]
x_train = train.iloc[:,0:13].values#,len(df.columns)-1]
print(x_train[0:2],y_train[0:2])
y_train.shape

lb = preprocessing.LabelBinarizer()
y_train = lb.fit_transform(y_train)
print(len(y_train))
print(y_train.shape)
print(y_train)

y_binary = to_categorical(y_train)
print(y_binary)
print(y_binary.shape)

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.33, shuffle= True)
print(len(x_train))
print(x_train.shape)
print(len(y_train))
print(y_train.shape)

from sklearn.preprocessing import StandardScaler

def dataPreProcessing(x):
    sc = StandardScaler()
    x = sc.fit_transform(x)
    print(x.shape)
    return x

x_train = dataPreProcessing(x_train)

    

[[5.20000000e+01 6.00000000e+01 5.87063800e+06 5.87177800e+06
  2.26044677e+03 2.40588372e+05 4.25313911e+00 5.17262850e+05
  5.87063800e+06 4.70235182e+05 1.02268473e+06 1.30000000e+01
  1.03453000e+07]
 [0.00000000e+00 7.00000000e+00 6.35242000e+05 6.35321000e+05
  2.27450335e+05 3.65836055e+03 2.73413302e+02 1.05025965e+04
  6.35042000e+05 5.61577974e+03 2.13475751e+04 0.00000000e+00
  1.49663530e+07]] [['CHAT']
 ['VIDEO-STREAMING']]
3360
(3360, 8)
[[0 0 1 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]
[[[1. 0.]
  [1. 0.]
  [0. 1.]
  ...
  [1. 0.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]
  ...
  [1. 0.]
  [0. 1.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [0. 1.]
  ...
  [1. 0.]
  [1. 0.]
  [1. 0.]]

 ...

 [[1. 0.]
  [1. 0.]
  [1. 0.]
  ...
  [1. 0.]
  [1. 0.]
  [0. 1.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]
  ...
  [1. 0.]
  [1. 0.]
  [0. 1.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]
  ...
  [1. 0.]
  [1. 0.]
  [0. 1.]]]
(3360, 8, 

In [144]:
    classifier = Sequential()
    #First Hidden Layer
    classifier.add(Dense(350, activation='relu', kernel_initializer='random_normal', input_shape=(len(fields)-1,1)))
    #Second  Hidden Layer
    classifier.add(Dense(350, activation='relu', kernel_initializer='random_normal'))
    #Third Hidden Layer
    classifier.add(Dense(350, activation='relu', kernel_initializer='random_normal'))
    #Output Layer
    classifier.add(Flatten())
    classifier.add(Dense(8, activation='sigmoid', kernel_initializer='random_normal'))
   
    #Compile the model
    
    #classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])
    classifier.compile(loss='binary_crossentropy', # Better loss function for neural networks
              optimizer=Adam(learning_rate=LEARN_RATE), # Adam optimizer with 1.0e-4 learning rate
                     metrics = ['accuracy'])
    classifier.summary()

Model: "sequential_45"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_182 (Dense)               │ (None, 13, 350)        │           700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_183 (Dense)               │ (None, 13, 350)        │       122,850 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_184 (Dense)               │ (None, 13, 350)        │       122,850 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_45 (Flatten)            │ (None, 4550)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_185 (Dense)               │ (None, 8)              │        36,408 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 282,808 (1.08 MB)

 Trainable params: 282,808 (1.08 MB)

 Non-trainable params: 0 (0.00 B)

In [145]:
history1 = classifier.fit(x_train, y_train,
                     epochs=20,
                    # validation_data=(x_valid,y_valid),
                     shuffle=True,
                   #  callbacks=[sgdr],
                     batch_size= 64)

Epoch 1/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.4787 - loss: 0.3400
Epoch 2/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.6384 - loss: 0.1921
Epoch 3/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.6752 - loss: 0.1787
Epoch 4/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.6666 - loss: 0.1747
Epoch 5/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.6977 - loss: 0.1683
Epoch 6/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.6993 - loss: 0.1640
Epoch 7/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.6987 - loss: 0.1639
Epoch 8/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.7431 - loss: 0.1507
Epoch 9/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.7381 - loss: 0.1507
Epoch 10/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.7162 - loss: 0.1530
Epoch 11/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.7449 - loss: 0.1470
Epoch 12/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy:

In [ ]:
sgdr = SGDR(min_lr=1e-5, max_lr = 1e-2, 
                     steps_per_epoch = np.ceil(len(x_train)/64),
                     lr_decay = 0.9,
                     cycle_length=10, 
                     mult_factor = 2)

In [196]:
model = Sequential()

model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(len(fields)-1,1),padding='same'))

model.add(Conv1D(filters=128, kernel_size=3, activation='relu',padding='same'))

#model.add(Dropout(0.5))

model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=256, kernel_size=3, activation='relu',padding='same'))

model.add(Conv1D(filters=256, kernel_size=3, activation='relu',padding='same'))

#model.add(Dropout(0.5))

model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dense(64, activation='relu'))

model.add(Dense(y_train.shape[1], activation='softmax'))

model.summary()

LEARN_RATE = 1.0e-4

model.compile(loss='categorical_crossentropy', # Better loss function for neural networks
              optimizer=Adam(learning_rate=1e-2), # Adam optimizer with 1.0e-4 learning rate
         
                     metrics = ['accuracy']) # Metrics to be evaluated by the model

Model: "sequential_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_98 (Conv1D)              │ (None, 14, 128)        │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_99 (Conv1D)              │ (None, 14, 128)        │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_48 (MaxPooling1D) │ (None, 7, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_100 (Conv1D)             │ (None, 7, 256)         │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_101 (Conv1D)             │ (None, 7, 256)         │       196,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_49 (MaxPooling1D) │ (None, 3, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_24 (Flatten)            │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_71 (Dense)                │ (None, 128)            │        98,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_72 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_73 (Dense)                │ (None, 6943)           │       451,295 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 903,199 (3.45 MB)

 Trainable params: 903,199 (3.45 MB)

 Non-trainable params: 0 (0.00 B)

### Correlation-based FSM #### 60sec

In [198]:
history1 = model.fit(x_train, y_train,
                     epochs=20,
                     #validation_data=(x_valid,y_valid),
                     shuffle=True,
                   #  callbacks=[sgdr],
                     batch_size= 64)

Epoch 1/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 5s 36ms/step - accuracy: 0.4921 - loss: 4.1867
Epoch 2/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.4974 - loss: 4.1229
Epoch 3/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - accuracy: 0.4956 - loss: 4.0936
Epoch 4/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - accuracy: 0.4995 - loss: 4.0500
Epoch 5/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - accuracy: 0.4861 - loss: 4.1301
Epoch 6/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - accuracy: 0.4940 - loss: 4.0591
Epoch 7/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - accuracy: 0.5048 - loss: 3.9245
Epoch 8/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - accuracy: 0.4933 - loss: 4.0004
Epoch 9/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 5s 36ms/step - accuracy: 0.4990 - loss: 4.0529
Epoch 10/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - accuracy: 0.4982 - loss: 3.9160
Epoch 11/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - accuracy: 0.4973 - loss: 4.2344
Epoch 12/20
147/147 ━━━━━━━━━━━━━━━━━━━━ 